In [7]:
initial_path = 'peptide-QML'
# initial_path = '..'

In [8]:
import sys
sys.path.append(initial_path)

%load_ext autoreload
%autoreload 2
from my_code import helper_classes as c
from my_code import pytorch_model as m
from my_code import quantum_nodes as q

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# name of the notebook
name_notebook = "sweep_6h_er_1qm_oi"

In [10]:
sweep = c.Sweep.load(file_name=name_notebook+"-1", day='1002', initial_path=initial_path)

In [11]:
for sweep_point in sweep.points_w_data:

    # print the point of the sweep
    sweep.print_sweep_status(sweep_point['idx'])

    # -- QUANTUM BLOCK -- #
    n_qubits = 6
    quantum_layer = q.circuit(
        n_qubits = n_qubits, 
        device = "default.qubit.torch",
        device_options = {'shots': None},
        embedding = q.parts.AngleEmbedding,
        embedding_ansatz = q.parts.Ansatz_11,
        block_ansatz = q.parts.Ansatz_11,
        final_ansatz = q.parts.Ansatz_11, 
        measurement = q.parts.Measurement('Z', 1),
        embedding_n_layers = sweep_point['embedding_dim']-1,
        different_inputs_per_layer = True,
        block_n_layers = sweep_point['block_n_layers'],
        # wrapper_qlayer = pw.QLayerEmpty,
    )

    # -- MODEL -- #                    
    class Model(m.Model):
        def __init__(self):
            super(Model, self).__init__()
            self.fc1 = m.nn.Embedding(num_embeddings=19, embedding_dim=sweep_point['embedding_dim'])
            self.fc2 = m.Reshape()
            self.fc3 = quantum_layer()            
            self.fc4 = m.nn.Linear(1, 1)

        def forward(self, x):
            for i in [1,2,3,4]:
                x = getattr(self, 'fc{}'.format(i))(x)
            return x

    model = Model()

    # set quantum layer
    model.set_quantum_layer(quantum_layer)

    # set the name and path of the model
    model.set_name_and_path(name_notebook, initial_path)

    #set sweep point
    model.set_sweep_point(
        sweep_uuid = sweep.uuid,
        sweep_point = sweep_point,
        day=sweep.day
    )
    
    # load results
    model.load_results(uuid=sweep_point['model_uuid'])

    # load best model
    model.load_state_dict_from_results(state_dict_name="parameters_epoch", rule="best")

    # -- GENERATE PEPTIDES and SAVE THEM -- #    
    oi_time = m.t.time()
    n_peptides = 20
    for i in range(n_peptides):
        model.get_input_with_low_score(
            layers_to_use = ['fc3', 'fc4'],
            n_iter = 1000,
            optimizer = m.torch.optim.Adam,
            optimizer_options = {'lr': 0.1},
            stop_criterion = 1e-3,
            initialization_seed = i,
            k_closest=2,
        )
        print("{}/{} peptides generated. Score {}. \t Time remaining: {:.2f} min \t\t\t ".format(
            i+1, n_peptides, model.optimized_inputs[-1][1], (n_peptides-i)*(m.t.time() - oi_time)/(i+1)/60
        ), end='\r')
    oi_time = m.t.time() - oi_time
    model.save_input_optimization_results()
    best_oi, best_oi_score = model.best_optimized_input
    print("Best score: {} \t\t\t\t\t\t  ".format(best_oi_score))

    # -- ADD DATA GENERATED TO SWEEP -- #
    sweep.add_data(
        idx=sweep_point['idx'],
        new_best_oi=best_oi,
        new_best_oi_score=best_oi_score,
        new_oi_time=oi_time,
    )
    sweep.save(csv=False) #TODO: save csv=False automatically when sweep is already saved ion csv

    



 --- SWEEP POINT 1/72: (1, 2, 3) ---
 --- parameters sweeping: ['repetitions', 'embedding_dim', 'block_n_layers'] 

Loaded results with uuid 95cf42c6-3d4e-4a46-a01a-247ffbbc25c8.
Loaded best state_dict from results.


Best score: -20.481258412586012 						                   20/20 peptides generated. Score -17.06491532196551. 	 Time remaining: 0.11 min 			 	 


 --- SWEEP POINT 2/72: (1, 2, 5) ---
 --- time reamining: 3518:10:19.81 

Loaded results with uuid d2e68efe-2b12-42eb-b759-bcd0cc3bce8a.
Loaded best state_dict from results.
Best score: -19.787385490421514 						                    20/20 peptides generated. Score -16.33545946558982. 	 Time remaining: 0.15 min 			  


 --- SWEEP POINT 3/72: (1, 2, 7) ---
 --- time reamining: 1736:03:19.77 

Loaded results with uuid d591d620-4ac4-4d6a-bb99-f2f2f7d75fbd.
Loaded best state_dict from results.
Best score: -17.041859430368053 						                 20/20 peptides generated. Score -9.18980028980488. 	 Time remaining: 0.18 min 			  			  


 --- SWEEP POINT 4/72: (1, 2, 9) ---
 --- time reamining: 1142:14:38.81 

Loaded results with uuid 80ecd4cb-19f4-43b0-a5dd-27b89285ccf2.
Loaded best state_dict from results.
Best score: -18.526346524977185 						    